In [ ]:
# Clean Jupyter environment
%reset -f

In [ ]:
from datetime import date, timedelta
import account
from garminworkouts.models.pace import Pace
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from performance.sample_performance import Races
vVO2 = Pace(account.vV02.pace).to_speed()
zones_mps = [zone * vVO2 for zone in [0.5, 0.65, 0.75, 0.87, 1.05, 1.2, 1.5, 1.8]]
zones_kph = [zone * 3.6 for zone in zones_mps]

In [ ]:

# Extract data from Races dictionary
delta = date.today() - timedelta(days=3*30)
race_data = {
    'dates': [Races[race][0] for race in Races if Races[race][0] > delta],
    'duration_secs': [Races[race][1] for race in Races if Races[race][0] > delta],
    'speed_mps': [Races[race][2] for race in Races if Races[race][0] > delta]
}
if not race_data['dates']:
    raise ValueError("No race data available. Execution stopped.")

# Create a DataFrame
df = pd.DataFrame(race_data)
df['duration_hours'] = df['duration_secs'] / 3600
df['duration_min'] = df['duration_secs'] / 60
df['speed_kph'] = df['speed_mps'] * 3.6

def VO2_test(race):
    date, time, speed = race
    distance = speed * time  # Calculate distance in meters
    time = time / 60  # Convert time to minutes
    v = distance / time
    VO2_test = -4.6 + 0.182258 * v + 0.000104 * v**2
    fVO2max = 0.8 + 0.1894393 * np.exp(-0.012778 * v) + 0.2989558 * np.exp(-0.1932605 * v)
    vVO2max = 1000 / (29.54 + 5.000663 * VO2_test - 0.007546 * VO2_test ** 2)
    return round(VO2_test)

In [ ]:
import matplotlib.pyplot as plt
from datetime import timedelta

def model(duration, s, e):
    return s * duration**(e - 1)

# Initial guess for the parameters
initial_guess = [0,1]

# Extract duration and speed data
duration = np.array(race_data['duration_secs'])
speed = np.array(race_data['speed_mps'])

# Fit the model to the data
params, covariance = curve_fit(f=model, xdata=duration, ydata=speed, p0=initial_guess)

# Extract the parameters
S,E = params
print(f"Fitted parameters: S = {S}, E = {E}")

In [ ]:
# Plot the fitted curve along with the data
d_min = 20 # 20 seconds
d_max = 3.5*3600 # hours to seconds
fit_duration_secs = np.linspace(d_min, d_max, 500)
fit_duration_min = fit_duration_secs / 60
fit_duration_hours = fit_duration_secs / 3600
fit_speed_mps = model(fit_duration_secs, S, E)
fit_speed_kph = fit_speed_mps * 3.6

# Convert duration to hours
ax = df.plot(x='duration_min', y='speed_kph', kind='scatter', zorder=5)
ax.set_xlabel('Time to exhaustion (min)')
ax.set_ylabel('Speed (km/h)')
ax.set_title('Time vs Speed')

ax.plot(fit_duration_min, fit_speed_kph, color='red', label='Fitted curve')

# Calculate the confidence intervals for the fitted parameters
sigma = np.sqrt(np.diag(covariance))
lower = params - sigma
upper = params + sigma

# Generate curves for the lower and upper confidence intervals
fit_speed_mps_lower = model(fit_duration_secs, *lower)
fit_speed_mps_upper = model(fit_duration_secs, *upper)
fit_speed_kph_lower = fit_speed_mps_lower * 3.6
fit_speed_kph_upper = fit_speed_mps_upper * 3.6

# Plot the confidence intervals
ax.fill_between(fit_duration_min, fit_speed_kph_lower, fit_speed_kph_upper,
                color='gray', alpha=0.2, label='95% Confidence Interval')
ax.legend()
# Add horizontal lines at zone values
for zone in zones_kph:
    if zone <= max(fit_speed_kph):
        ax.axhline(y=zone, color='gray', linestyle='--', linewidth=0.5)

In [ ]:
# Compute the integral of the fit curve
fit_integral_speed_m = fit_speed_mps * fit_duration_secs
fit_integral_speed_m_lower = fit_speed_mps_lower  * fit_duration_secs
fit_integral_speed_m_upper = fit_speed_mps_upper * fit_duration_secs

# Plot the integral of the fit curve
fig, ax2 = plt.subplots()
# Convert duration to hours
fit_integral_speed_km = fit_integral_speed_m / 1000
fit_integral_speed_km_lower = fit_integral_speed_m_lower / 1000
fit_integral_speed_km_upper = fit_integral_speed_m_upper / 1000

# Plot the integral of the fit curve
ax2.plot(fit_integral_speed_km, fit_duration_min, label='Fitted Curve', color='red')
ax2.set_ylabel('Duration (min)')
ax2.set_xlabel('Distance (km)')
ax2.set_title('Prognostics')

# Plot the confidence intervals
ax2.plot(fit_integral_speed_km_lower, fit_duration_min, label='Min', linestyle='--')
ax2.plot(fit_integral_speed_km_upper, fit_duration_min, label='Max', linestyle='--')
ax2.legend()

# Plot the distances 21.1km and 10km on the curve
pred_distances_km = [5, 10, 21.1, 42.2]
pred_distances_m = pred_distances_km * 1000
pred_distances_m = [d * 1000 for d in pred_distances_km]
pred_durations_secs = np.interp(pred_distances_m, fit_integral_speed_m, fit_duration_secs)
pred_durations_hours = pred_durations_secs / 3600
pred_durations_min = pred_durations_secs / 60
pred_durations_hours_hms = [str(timedelta(seconds=s))for s in pred_durations_secs]

# Add intermittent lines from the scatter points to X and Y axis
for i, txt in enumerate(pred_distances_km):
    ax2.annotate(f'{txt} km', (pred_distances_km[i], pred_durations_min[i]),
                 textcoords="offset points", xytext=(-25, 5), ha='center')
    ax2.scatter(pred_distances_km, pred_durations_min, color='blue', zorder=5)
    ax2.axhline(y=pred_durations_min[i], color='gray', linestyle='--',
                linewidth=0.5)

# Show the plot after adding scatter and annotations
plt.show()